# Load Libraries & Dataframe

In [ ]:
# Manipulation
import os
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
!pip install streamlit
import streamlit as st
import plotly.express as px
import missingno as msno

# Regression, Amputation, & Testing
from scipy import stats
from scipy.stats import skew, boxcox, mstats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures, FunctionTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
github_raw_csv_url = "https://raw.githubusercontent.com/NitoBoritto/Car_Sales_Regression_Analysis/master/car_resale_prices.csv"
Car_df = pd.read_csv(github_raw_csv_url)
Car_df

# EDA

##Initial Data Inspection

In [ ]:
Car_df.shape

In [ ]:
Car_df.info()

In [ ]:
Car_df["full_name"].nunique()

In [ ]:
Car_df["insurance"].unique()

In [ ]:
Car_df["transmission_type"].unique()

In [ ]:
Car_df["owner_type"].unique()

In [ ]:
Car_df["fuel_type"].unique()

In [ ]:
Car_df["body_type"].unique()

In [ ]:
Car_df.head()

In [ ]:
import missingno as msno
msno.matrix(Car_df)

In [ ]:
sns.pairplot(Car_df)

## Categorical Feature Analysis

Fuel Type vs Resale Price

In [ ]:
plt.figure(figsize=(12,10))
sns.set_theme(style="whitegrid")
sns.boxplot(x='fuel_type', y='resale_price', data=Car_df, color='green')
plt.yscale('log')
plt.title('Resale Price by Fuel Type')
plt.show()

Transmission vs Resale Price

In [ ]:
# Count transmission types for pie chart
transmission_counts = Car_df['transmission_type'].value_counts()

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(18, 10))

# Boxplot
sns.boxplot(
    x='transmission_type',
    y='resale_price',
    data=Car_df,
    color='lightgreen',
    ax=axes[0]
)
axes[0].set_yscale('log')
axes[0].set_title('Resale Price by Transmission')

# Pie chart
axes[1].pie(
    transmission_counts,
    labels=transmission_counts.index,
    autopct='%1.1f%%',
    startangle=90,
    colors=['#c7e9c0', '#a1d99b', '#74c476', '#41ab5d']
)
axes[1].set_title('Transmission Type Distribution')
axes[1].axis('equal')

plt.tight_layout()
plt.show()

Owner Type vs Resale Price

In [ ]:
plt.figure(figsize=(12,10))
sns.boxplot(
    x='owner_type',
    y='resale_price',
    data=Car_df,
    color='#74c476'
)
plt.yscale('log')
plt.title('Resale Price by Owner Type')
plt.show()

In [ ]:
owner_counts = Car_df['owner_type'].value_counts()

# Create donut chart
plt.figure(figsize=(8,8))
plt.pie(
    owner_counts,
    labels=owner_counts.index,
    autopct='%1.1f%%',
    startangle=90,
    colors=['#c7e9c0', '#a1d99b', '#74c476', '#41ab5d'],
    wedgeprops={'width': 0.4}  # creates the donut hole
)

plt.title('Distribution of Owner Types')
plt.axis('equal')  # keep circle shape
plt.show()

##Correlation Analysis

In [ ]:
numeric_cols = Car_df.select_dtypes(include=['float64','int64','int32'])
plt.figure(figsize=(8,6))
sns.heatmap(numeric_cols.corr(), annot=True, cmap='Greens', fmt=".2f", linewidths=0.2)
plt.title("Correlation Heatmap")
plt.show()

##resale_price_column

 we make a histogram to see the the distribution of the price , we relaize that there is right skewness

In [ ]:
plt.figure(figsize=(12,10))
plt.hist(Car_df['resale_price'], bins=50, color='#73A580')
plt.show()

In [ ]:
Car_df['resale_price'].describe()

##column seets

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(x='seats', data=Car_df, color='lightgreen')
plt.title('Distribution of Car Seats')
plt.xlabel('Number of Seats')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))

sns.barplot(x='body_type', y='resale_price', data=Car_df, color='green')

plt.title('Resale Price by Body Type')
plt.ylabel('Resale Price')
plt.xlabel('Body Type')
plt.show()

# Preprocessing

In [ ]:
Car_df = Car_df.drop(columns="Unnamed: 0")
Car_df

In [ ]:
Car_df.info()

In [ ]:
Missing_Summary=pd.DataFrame({
    'Number Of Nulls':Car_df.isna().sum(),
    'Nulls Precentage':round((Car_df.isna().sum()/len(Car_df))*100,2)
})
Missing_Summary

## registered_year

In [ ]:
Car_df['registered_year'].unique()

In [ ]:
Car_df['registered_year']=Car_df['registered_year'].str.strip(". JanFebMarAprMayJunJulAugSepOctNovDec")

In [ ]:
Car_df['registered_year_Imputation']=Car_df['full_name'].str[:4].astype(int)
Car_df

In [ ]:
Car_df[Car_df['registered_year'].isnull()]

In [ ]:
Car_df['registered_year_Imputation']=Car_df['full_name'].str[:4].astype(int)
Car_df

In [ ]:
Car_df['registered_year'][Car_df['registered_year'].isnull()] = Car_df['registered_year_Imputation'][Car_df['registered_year'].isnull()].astype(int)


In [ ]:
Car_df['registered_year']=Car_df['registered_year'].astype(int)

In [ ]:
Car_df['registered_year'].unique()

In [ ]:
Car_df

## engine_capacity

In [ ]:
Car_df['engine_capacity']=Car_df['engine_capacity'].str.strip("c")
Car_df['engine_capacity']=Car_df['engine_capacity'].astype(float)
Car_df['engine_capacity']=Car_df['engine_capacity'].replace([0,'   nan'],np.nan)


In [ ]:
Car_df['engine_capacity'].unique()

In [ ]:
plt.figure(figsize= (12,8))
sns.histplot(data = Car_df,x='engine_capacity')

In [ ]:
Car_df['engine_capacity'].unique()

In [ ]:
# Missing values imputation
Car_df['engine_capacity']=Car_df['engine_capacity'].fillna(Car_df['engine_capacity'].median()).astype(int)

In [ ]:
Car_df['engine_capacity'].unique()

## Resale_Price

In [ ]:
sns.histplot(Car_df['resale_price'])
plt.show()

In [ ]:
# resale_price ->(Lakh = 100,000)
Car_df

In [ ]:
Car_df['resale_price'] = Car_df['resale_price'].str.replace(',', '')
Car_df['resale_price'] = Car_df['resale_price'].str.strip("₹ LakhCrore")
Car_df['resale_price'] = Car_df['resale_price'].astype(float)

In [ ]:
Car_df['resale_price']=Car_df['resale_price']*100000
Car_df['resale_price']=Car_df['resale_price']
Car_df['resale_price']=Car_df['resale_price']*0.01111784
Car_df['resale_price']=Car_df['resale_price'].round(2)
#Car_df.rename(columns={'resale_price':'resale_price in (Dollars)'})

In [ ]:
Car_df

## **Kms_driven**

In [ ]:
Car_df[Car_df['kms_driven'].isnull()]

In [ ]:
Car_df['kms_driven']=Car_df['kms_driven'].str.strip("Kms ,")
Car_df['kms_driven']=Car_df['kms_driven'].str.replace(",","")
Car_df['kms_driven']=Car_df['kms_driven'].astype(float)
Car_df['kms_driven']=Car_df['kms_driven'].fillna(Car_df['kms_driven'].median())
Car_df['kms_driven']=Car_df['kms_driven'].astype(int)


In [ ]:
Car_df['kms_driven'].unique()

In [ ]:
Car_df

In [ ]:
Car_df['kms_driven']

In [ ]:
Car_df.info()

## insurance


In [ ]:
Car_df

In [ ]:
Car_df['insurance'].unique()

In [ ]:
insurance={
    'Third Party insurance':"Third Party",
    '1': np.nan,
    '2': np.nan,
    'Not Available':np.nan
}
Car_df['insurance']=Car_df['insurance'].replace(insurance)
Car_df['insurance']=Car_df['insurance'].replace(np.nan,"No Insurance")

In [ ]:
Car_df['insurance'].unique()

## Owner Type

In [ ]:
Car_df['owner_type'].unique()

In [ ]:
Car_df[Car_df['owner_type'].isnull()][['kms_driven']]
print(Car_df[Car_df['owner_type'].isnull()][['kms_driven']].describe())

In [ ]:
bins = [0, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000,
        50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000,
        100000, float('inf')]
labels = [
    '0-5k', '5k-10k', '10k-15k', '15k-20k', '20k-25k', '25k-30k',
    '30k-35k', '35k-40k', '40k-45k', '45k-50k', '50k-55k', '55k-60k',
    '60k-65k', '65k-70k', '70k-75k', '75k-80k', '80k-85k', '85k-90k',
    '90k-95k', '95k-100k', '100k+']
Car_df['km_group'] = pd.cut(Car_df['kms_driven'], bins=bins, labels=labels)
relation = pd.crosstab(Car_df['km_group'], Car_df['owner_type'])
relation

In [ ]:
bins = [0, 20000, 60000, 100000, float('inf')]
labels = ['0-20k', '20k-60k', '60k-100k', '100k+']
Car_df['km_group'] = pd.cut(Car_df['kms_driven'], bins=bins, labels=labels)
relation = pd.crosstab(Car_df['km_group'], Car_df['owner_type'])
print(relation)

In [ ]:
Df_missing=Car_df[Car_df['owner_type'].isnull()]
Df_missing

In [ ]:
bins=[0,20000,60000,100000,float('inf')]
labels=['0-20k','20k-60k','60k-100k','100k+']
Df_missing.groupby(pd.cut(Df_missing['kms_driven'],bins=bins,labels=labels)).size()

In [ ]:
Car_df['km_group']=Car_df['km_group'].astype(str)

In [ ]:
Car_df['km_group'].unique()

In [ ]:
Car_df[Car_df['km_group']=="0-20k"].isna().sum()

In [ ]:
Car_df.loc[
    (Car_df['owner_type'].isnull()) & (Car_df['km_group'] == '0-20k'),
    'owner_type']

In [ ]:
Car_df.loc[
    (Car_df['owner_type'].isnull()) & (Car_df['km_group'] == '0-20k'),
    'owner_type']='First Owner'

In [ ]:
Car_df[Car_df['km_group']=="20k-60k"].isna().sum()

In [ ]:
Car_df['owner_type'].unique()

In [ ]:
Car_df.loc[
    (Car_df['owner_type'].isnull()) & (Car_df['km_group'] == '20k-60k'),
    'owner_type']='Second Owner'

In [ ]:
Car_df[Car_df['km_group']=="60k-100k"].isna().sum()

In [ ]:
Car_df.loc[
    (Car_df['owner_type'].isnull()) & (Car_df['km_group'] == '60k-100k'),
    'owner_type']='Third Owner'

In [ ]:
Car_df[Car_df['km_group']=="100k+"].isna().sum()

In [ ]:
Car_df.loc[
    (Car_df['owner_type'].isnull()) & (Car_df['km_group'] == '100k+'),
    'owner_type']='Fifth Owner'

In [ ]:
Car_df['owner_type'].unique()

## max_power



In [ ]:
Car_df['max_power']=Car_df['max_power'].str.strip(" ")
Car_df['max_power']=Car_df['max_power'].replace(".","")

In [ ]:
Car_df['max_power'].unique()

In [ ]:
mp = Car_df["max_power"].astype(str)

In [ ]:
is_ps = mp.str.contains("PS", case=False, na=False)

In [ ]:
max_power_numeric = mp.str.extract(r"([0-9]*\.?[0-9]+)")[0].astype(float)

In [ ]:
max_power_numeric[is_ps] = max_power_numeric[is_ps] * 0.98632

In [ ]:
Car_df["max_power_bhp"] = max_power_numeric

In [ ]:
Car_df["max_power_bhp"] = Car_df["max_power_bhp"].fillna(Car_df["max_power_bhp"].median())

In [ ]:
Car_df

In [ ]:
Car_df['max_power'].unique()

In [ ]:
import re

#1) Standardize text
s = Car_df["max_power"].astype(str).str.strip()
s_lower = s.str.lower()

# Helper: first number in string
def get_first_number(x):
    m = re.search(r"[0-9].?[0-9]+", x)
    return float(m.group()) if m else np.nan

clean_vals = []

for txt, low in zip(s, s_lower):
    v = get_first_number(txt)      # base numeric
    if np.isnan(v):
        clean_vals.append(np.nan)
        continue

    # If PS → convert PS to bhp
    if "ps" in low:
        v = v * 0.98632

    # If kW → convert kW to bhp
    if "kw" in low:
        v = v * 1.34102

    # If bhp or no unit → keep v as bhp
    clean_vals.append(v)

#2) Create clean column
clean_series = pd.Series(clean_vals)

#3) Fill missing with median so NOTHING is missing
median_val = clean_series.median()
clean_series = clean_series.fillna(median_val).round(2)

Car_df["max_power_clean_bhp"] = clean_series

In [ ]:
Car_df


In [ ]:
Car_df.drop('max_power_bhp', axis=1, inplace=True)
Car_df.drop('max_power', axis=1, inplace=True)

In [ ]:
Car_df

## Seats Column

In [ ]:
median=Car_df['seats'].median()
Car_df['seats']=Car_df['seats'].fillna(median)
Car_df['seats']=Car_df['seats'].astype(int)

In [ ]:
Car_df


In [ ]:
Car_df['seats'].unique()

## Mileage


In [ ]:
Car_df[['mileage_number', 'mileage_unit']] = Car_df['mileage'].str.split(' ', expand=True)
Car_df['mileage_number'] = Car_df['mileage_number'].astype(float)
Car_df[['mileage', 'mileage_number', 'mileage_unit']].head()

In [ ]:
Car_df[Car_df['mileage_number'].isna()]

In [ ]:
Car_df['mileage_unit'].unique()

In [ ]:
sns.histplot(Car_df['mileage_number'])
plt.show()

KNN Milage_Number

In [ ]:
# 1. Separate your data: Keep only numbers for the imputer
numeric_cols = Car_df.select_dtypes(include=['int64', 'int32' ,'float64']).columns
data_for_imputing = Car_df[numeric_cols]

# 2. Initialize and run the imputer
imputer = KNNImputer(n_neighbors=5)
imputed_array = imputer.fit_transform(data_for_imputing)

# 3. Put the results back into the original dataframe
Car_df[numeric_cols] = imputed_array

# Check results
print(Car_df['mileage_number'].isnull().sum())

KNN Milage_Unit

In [ ]:
# 1. Label Encode 'mileage_unit' (while preserving NaNs)
le = LabelEncoder()
mask = Car_df['mileage_unit'].notnull()

# Create a temporary encoded column     (0	km/kg , 1	km/l)
Car_df.loc[mask, 'mileage_unit_encoded'] = le.fit_transform(Car_df.loc[mask, 'mileage_unit'])

# 2. Select ONLY numeric columns for the imputer
# We exclude the original string columns to avoid the "could not convert string to float" error.
numeric_cols = Car_df.select_dtypes(include=['number']).columns.tolist()

# 3. Scaling (Essential for KNN)
# This ensures columns like 'year' and 'mileage_number' are on the same scale (0 to 1)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(Car_df[numeric_cols])

# 4. Initialize and Run KNNImputer
imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(scaled_data)

# 5. Reverse the scaling to get original values back
final_numeric_data = pd.DataFrame(scaler.inverse_transform(imputed_data), columns=numeric_cols)

# 6. Final Step: Decode 'mileage_unit_encoded' back to strings
# Since KNN returns averages (e.g. 0.4), we round to the nearest integer (0 or 1)
unit_indices = final_numeric_data['mileage_unit_encoded'].round().astype(int)
Car_df['mileage_unit'] = le.inverse_transform(unit_indices)

# (Optional) Update mileage_number as well if you want both imputed
Car_df['mileage_number'] = final_numeric_data['mileage_number']

# Drop the temporary encoded column
Car_df = Car_df.drop(columns=['mileage_unit_encoded'])

print("Missing values after imputation:")
print(Car_df[['mileage_unit', 'mileage_number']].isnull().sum())

In [ ]:
Missing_Summary=pd.DataFrame({
    'Number Of Nulls':Car_df.isna().sum(),
    'Nulls Precentage':round((Car_df.isna().sum()/len(Car_df))*100,2)
})
Missing_Summary

## Body_Type

In [ ]:
Car_df['body_type'].unique()

In [ ]:
Car_df

In [ ]:
# Show total number of rows
print("Total rows:", len(Car_df))

# Define real body types
real_body_types = ["Hatchback", "MUV", "Sedan", "Minivans", "SUV", "Coupe", "Pickup","Convertibles"]

mask_real = Car_df["body_type"].isin(real_body_types)
percentage_real = mask_real.mean() * 100

print(f"Percentage of real body types: {percentage_real:.2f}%")

# Print all rows (careful: this can be very long)
print(Car_df.to_string())

In [ ]:
# Map real body types to 1..n
mapping = {btype: i + 1 for i, btype in enumerate(real_body_types)}

# Encode: real body types → 1..n, others → 0
Car_df["body_type_encoded"] = Car_df["body_type"].map(mapping).fillna(0).astype(int)

print(Car_df[["body_type", "body_type_encoded"]].drop_duplicates())

In [ ]:
# Real body types and encoding (as before)
real_body_types = ["Hatchback", "MUV", "Sedan", "Minivans", "SUV", "Coupe", "Pickup", "Convertibles"]
mapping = {btype: i + 1 for i, btype in enumerate(real_body_types)}
Car_df["body_type_encoded"] = Car_df["body_type"].map(mapping).fillna(0).astype(int)

# Count rows per body_type
counts = Car_df["body_type"].value_counts().rename("body_type_count")

# One row per body_type with its code and count
result = (
    Car_df[["body_type", "body_type_encoded"]]
    .drop_duplicates()
    .merge(counts, left_on="body_type", right_index=True, how="left")
    .sort_values("body_type_encoded")
)

print(result)

In [ ]:
Car_df['body_type_encoded'] = Car_df['body_type_encoded'].replace(0, np.nan)

In [ ]:
imputer = KNNImputer(n_neighbors=5)
Car_df['body_type_encoded'] = imputer.fit_transform(Car_df[['body_type_encoded']]).astype(int)

print("Unique values in body_type_encoded after imputation:", Car_df['body_type_encoded'].unique())
print("Null values in body_type_encoded after imputation:", Car_df['body_type_encoded'].isnull().sum())

In [ ]:
reverse_mapping = {i + 1: btype for i, btype in enumerate(real_body_types)}

# Apply the reverse mapping to create a temporary string column
Car_df['body_type_decoded'] = Car_df['body_type_encoded'].map(reverse_mapping)

Car_df['body_type'] = Car_df['body_type_decoded']

# Drop the 'body_type_encoded' and 'body_type_decoded' columns
Car_df.drop(columns=['body_type_encoded', 'body_type_decoded'], inplace=True)

print("Unique values in 'body_type' after decoding:", Car_df['body_type'].unique())

## Duplicate

In [ ]:
Car_df.duplicated().sum()

In [ ]:
Car_df.shape

In [ ]:
Car_df.drop_duplicates(inplace=True)


In [ ]:
Car_df.shape

In [ ]:
Car_df.duplicated().sum()

In [ ]:
Car_df.dropna(subset=['mileage'], inplace=True)

In [ ]:
Car_df.head()

## Outliers

In [ ]:
numeric_cols = Car_df.select_dtypes(include=['float64','int64','int32'])
plt.figure(figsize=(8,6))
sns.heatmap(numeric_cols.corr(),
            annot=True,
            cmap='Greens',
            fmt=".2f",
            linewidths=0.2)
plt.title("Correlation Heatmap Before Transformation")
plt.xticks(rotation = 45, ha = "right")
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
for i, num in enumerate(numeric_cols, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(Car_df[num])
    plt.title(f"Histogram of {num}")
plt.tight_layout()
plt.show()

In [ ]:
outlie = Car_df.select_dtypes(include=["float64", "int64", "int32"]).drop(columns=["registered_year", "seats", "registered_year_Imputation"])
outlie_winsorized = outlie.copy()

# Winsorize the column at the 5th and 95th percentiles
for col in outlie_winsorized.columns:
    outlie_winsorized[col] = mstats.winsorize(outlie_winsorized[col], limits=[0.05, 0.05])

# Update Car_df with the winsorized numerical columns
for col in outlie_winsorized.columns:
    Car_df[col] = outlie_winsorized[col]

print("Boxplots after Winsorization:")
plt.figure(figsize=(12, 10))
for i, num in enumerate(outlie_winsorized.columns, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(Car_df[num])
    plt.title(f"Boxplot of {num} (Winsorized)")
plt.tight_layout()
plt.show()

In [ ]:
boxcox_resale_price, fitted_lambda = boxcox(Car_df["resale_price"])

print(f"Best lambda for the job: {fitted_lambda}")

Car_df['boxcox_resale_price'] = boxcox_resale_price
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

sns.boxplot(Car_df['boxcox_resale_price'], ax = ax1)
ax1.set_title('Boxcox Transformed Resale Price')

plt.figure(figsize= (12,8))
sns.histplot(Car_df['boxcox_resale_price'],
             kde = True,
             ax = ax2)
ax2.set_title('Boxcox Transformed Resale Price')

print(f"Skewness after transformation: {Car_df["boxcox_resale_price"].skew()}")
print(f"Kurtosis after transformation: {Car_df["boxcox_resale_price"].kurtosis()}")

In [ ]:
numeric_cols = Car_df.select_dtypes(include=['float64','int64','int32'])
plt.figure(figsize=(8,6))
sns.heatmap(numeric_cols.corr(),
            annot=True,
            cmap='Greens',
            fmt=".2f",
            linewidths=0.2)
plt.title("Correlation Heatmap After Transformation")
plt.xticks(rotation = 45, ha = "right")
plt.show()

In [ ]:
Car_df.to_csv("car_data_cleaned.csv", index=False)
print("✅ Cleaned data saved to 'car_data_cleaned.csv'")

## One Way Anova

In [ ]:
sns.boxplot(x='owner_type', y='boxcox_resale_price', data=Car_df)
plt.title('Checking Spread and Outliers by owner_type')
plt.show()

 H_0 Null Hypothesis : All owners have the same average price any difference is just a fluke.

 H_1 Alternative Hypothesis : at least one group  has a significantly different price than the others

In [ ]:
import statsmodels.api as sm # used for estimating and testing statistical models.
from statsmodels.formula.api import ols # used to fit the linear model that ANOVA relies on.
model = ols('boxcox_resale_price ~ owner_type', data=Car_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

anova_table

We reject H_0 the idea that all owner types have the same price

Meaning :The price gaps you see between a "First Owner" and a "Third Owner" are not random flukes or errors in your data

## Spearman’s Rank Correlation [Owner_Type (Ordinal)]

In [ ]:

owner_map = {'First Owner': 1, 'Second Owner': 2, 'Third Owner': 3, 'Fourth Owner': 4, 'Fifth Owner': 5}
Car_df['owner_rank'] = Car_df['owner_type'].map(owner_map)
correlation = Car_df['owner_rank'].corr(Car_df['boxcox_resale_price'], method='spearman')
print(f"Correlation: {correlation:.4f}")

# EDA

In [ ]:
msno.matrix(Car_df)

## resale_price coulmn

In [ ]:

plt.figure(figsize=(18, 10))

# --- 1. Histogram of resale_price (NO transformation) ---
plt.subplot(2, 2, 1)
sns.histplot(Car_df['resale_price'], bins=50)
plt.title("Resale Price Distribution (Original)")
plt.xlabel("Resale Price")
plt.ylabel("Count")

# --- 2. Log-transformed histogram ---
plt.subplot(2, 2, 2)
sns.histplot(np.log1p(Car_df['resale_price']), bins=50, kde=True)
plt.title("Log-Transformed Resale Price")
plt.xlabel("log(1 + Resale Price)")

# --- 3. Violin plot of resale_price ---
plt.subplot(2, 2, 3)
sns.violinplot(x=Car_df['resale_price'], inner="quartile")
plt.title("Violin Plot of Resale Price")

# --- 4. Violin plot seats vs resale_price (log scale) ---
plt.subplot(2, 2, 4)
sns.violinplot(x='seats', y='resale_price', data=Car_df)
plt.yscale('log')
plt.title("Resale Price by Seats (Log Scale)")

plt.tight_layout()
plt.show()


After we transform the resale price by taking log , we see that the column become normal distributed but stil there some outlier

## registred_year_column

In [ ]:
year_df = Car_df.groupby("registered_year")["resale_price"].agg(["mean","sum"]).reset_index()
fig , ax = plt.subplots(2,1,figsize = (20,12))
ax1 = sns.pointplot(x=year_df["registered_year"],y=year_df["sum"],ax=ax[0])
ax1.set_ylabel("Sum of Sales")
ax2 = sns.pointplot(x=year_df["registered_year"],y=year_df["mean"],ax=ax[1],color= "r")
ax2.set_ylabel("Mean of Sales")
plt.tight_layout()

Cars registered in these years have the highest average resale value in your dataset. This could suggest these specific years contain high-value luxury or collector cars that skew the average upward

## engine_capacity_column

after we take the log the skewness is reduced and the the graph become more normal

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats


fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# 1. Square Root Transformation
axes[0].hist(np.sqrt(Car_df['engine_capacity']), bins=50, color='green', edgecolor='black')
axes[0].set_title(r'$\sqrt{\text{Engine Capacity}}$ Distribution')
axes[0].set_xlabel(r'$\sqrt{\text{Engine Capacity}}$')
axes[0].set_ylabel('Count')

# 2. Log Transformation (log1p handles 0 values if present)
axes[1].hist(np.log1p(Car_df['engine_capacity']), bins=50, color='green', edgecolor='black')
axes[1].set_title(r'$\log(1+x)$ Transformed Engine Capacity')
axes[1].set_xlabel(r'$\log(\text{Engine Capacity})$')
axes[1].set_ylabel('Count')

# 3. Box-Cox Transformation

boxcox_data, lambda_value = stats.boxcox(Car_df['engine_capacity'])
axes[2].hist(boxcox_data, bins=50, color='green', edgecolor='black')
axes[2].set_title(f'Box-Cox Transformed ($\lambda = {lambda_value:.4f}$)')
axes[2].set_xlabel('Box-Cox Transformed Value')
axes[2].set_ylabel('Count')

# Adjust layout to prevent label overlapping
plt.tight_layout()

# Save the final combined plot
plt.savefig('engine_capacity_transformations.png')

In [ ]:
plt.figure(figsize=(6, 8))

sns.boxplot(y=Car_df['engine_capacity'])

plt.title('Box Plot of Engine Capacity')
plt.ylabel('Engine Capacity (units)')
plt.show()

## km_driven

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(np.log1p(Car_df['kms_driven']), bins=50, kde=True)

plt.title('Log-Transformed Kms Driven Distribution')
plt.xlabel('log(Kms Driven)')
plt.ylabel('Count')
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot((Car_df['km_group']), bins=50, kde=True)

plt.title(' Kms group Distribution')
plt.xlabel('(Km group)')
plt.ylabel('Count')
plt.show()

after we take the log the skewness is reduced and the the graph become more normal

## max power

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot((Car_df['max_power_clean_bhp']), bins=50, kde=True)

plt.title('max power Distribution')
plt.xlabel('(max power)')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(np.log1p(Car_df['max_power_clean_bhp']), bins=50, kde=True)

plt.title('Log-Transformed max power Distribution')
plt.xlabel('log(max power)')
plt.ylabel('Count')
plt.show()

## mileage

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot((Car_df['mileage_number']), bins=50, kde=True)

plt.title('mileage Distribution')
plt.xlabel('(mileage)')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(np.log1p(Car_df['mileage_number']), bins=50, kde=True)

plt.title('log transformation mileage Distribution')
plt.xlabel('log(mileage)')
plt.ylabel('Count')
plt.show()

## fuel_type

In [ ]:
Car_df['fuel_type'].value_counts().plot(kind='bar')
plt.xlabel('fuel_type')
plt.ylabel('Count')
plt.title('Distribution of fuel_type')
plt.show()

## owner_type

In [ ]:
Car_df['owner_type'].value_counts().plot(kind='bar')
plt.xlabel('owner_type')
plt.ylabel('Count')
plt.title('Distribution of owner_type')
plt.show()

## transmission_type

In [ ]:
Car_df['transmission_type'].value_counts().plot(kind='bar')
plt.xlabel('transmission_type')
plt.ylabel('Count')
plt.title('Distribution of transmission_type')
plt.show()

## body_type

In [ ]:
Car_df['body_type'].value_counts().plot(kind='bar')
plt.xlabel('body_type')
plt.ylabel('Count')
plt.title('Distribution of body_type')
plt.show()

# Regression

In [ ]:
Car_df.info()

## Transformation

In [ ]:
categorical_cols = Car_df.select_dtypes(include= "object").drop(columns = ["full_name", "mileage"])

for col in  categorical_cols:
    unique_vals = [Car_df[col].unique()]
    print(f"Unique values in {col}: {unique_vals}")

Creating Dummies

In [ ]:
Car_df = pd.get_dummies(Car_df, columns=['insurance', 'owner_type', 'fuel_type', 'body_type'],
                                         drop_first = True,
                                         dtype = int)

In [ ]:
label_encoder = LabelEncoder()

Car_df['transmission_type_encoded'] = label_encoder.fit_transform(Car_df['transmission_type'])

In [ ]:
Car_df.info()

## Model development (Linear only)

ML Regression

In [ ]:
numerical_cols = Car_df.select_dtypes(include=['float64','int64','int32']).drop(columns = ["boxcox_resale_price", "resale_price", "registered_year_Imputation", "price_per_km", "owner_rank"]).columns
scaler = StandardScaler()

X = Car_df[numerical_cols]
X_scaled = scaler.fit_transform(X)

X_vif = pd.DataFrame()
X_vif["features"] = X.columns
X_vif["VIF"] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]

X_vif.sort_values("VIF", ascending=False)

In [ ]:
y = Car_df[['boxcox_resale_price']]

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
ml = LinearRegression()
ml.fit(X_train, y_train)
predictions = ml.predict(X_test)

print("Mean Squared Error of only ML: ", mean_squared_error(y_test, predictions))
print("R-Squared of only ML", r2_score(y_test, predictions))

Monte Carlo

In [ ]:
n_runs = 100
r2_scores = []
mse_scores = []

for i in range(n_runs):
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled,
        y,
        test_size = 0.3,
        random_state = i
    )
    ml = LinearRegression()
    ml.fit(X_train, y_train)

    predictions = ml.predict(X_test)

    r2 = r2_score(y_test, predictions)
    r2_scores.append(r2)

    mse = mean_squared_error(y_test, predictions)
    mse_scores.append(mse)

average_r2_score = np.mean(r2_scores)

print(f"Average MSE over 100 iterations: {np.mean(mse_scores):.4f}")
print(f"Average R-Squared over 100 iterations: {average_r2_score:.4f}")

## Model development (Linear & Elastic Net)

Elastic Net Regression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

elastic = ElasticNetCV(
    l1_ratio=[.1, .3, .5, .7, .9, 1],  # mix between L1 and L2
    alphas=np.logspace(-3, 2, 50),    # candidate regularization strengths
    cv=5,
    random_state=42,
    max_iter=5000
)

elastic.fit(X_train, y_train)

# Best hyperparameters
print("Best alpha:", elastic.alpha_)
print("Best l1_ratio:", elastic.l1_ratio_)

# Coefficients
coef_df = pd.DataFrame({
    "feature": X.columns,
    "coefficient": elastic.coef_,
    "abs_coefficient": abs(elastic.coef_)
}).sort_values(by="abs_coefficient", key=abs, ascending=False)

print(coef_df)


ML Regression

In [ ]:
non_zero_features = X.columns[elastic.coef_ != 0]

X_selected = Car_df[non_zero_features]
X_selected_scaled = scaler.fit_transform(X_selected)

X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(X_selected_scaled,
                                                                                        y,
                                                                                        test_size=0.3,
                                                                                        random_state=42)
ml = LinearRegression()
ml.fit(X_train_selected, y_train_selected)
predictions = ml.predict(X_test_selected)

print("Mean Squared Error of ML with Elastic Net: ", mean_squared_error(y_test_selected, predictions))
print("R-Squared of ML with Elastic Net", r2_score(y_test_selected, predictions))

### Calculate Average R-squared over 100 Iterations



Monte Carlo

In [ ]:
n_runs = 100
r2_scores = []
mse_scores = []

for i in range(n_runs):
    X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(
        X_selected_scaled,
        y,
        test_size = 0.3,
        random_state = i
    )
    ml = LinearRegression()
    ml.fit(X_train_selected, y_train_selected)

    predictions = ml.predict(X_test_selected)

    r2 = r2_score(y_test_selected, predictions)
    r2_scores.append(r2)

    mse = mean_squared_error(y_test_selected, predictions)
mse_scores.append(mse)

average_r2_score = np.mean(r2_scores)

print(f"Average MSE over 100 iterations: {np.mean(mse_scores):.4f}")
print(f"Average R-Squared over 100 iterations: {average_r2_score:.4f}")

## Linearity Assumption

### Weighted Least Squares (WLS)

In [ ]:
X_sm = sm.add_constant(X_selected_scaled)

# Fit an OLS model to get initial residuals
ols_model = sm.OLS(y, X_sm)
ols_results = ols_model.fit()

# Calculate weights: a common heuristic is to use the inverse of the absolute residuals
weights = 1 / (np.abs(ols_results.resid) + 1e-6)

# Fit the WLS model using the calculated weights
wls_model = sm.WLS(y, X_sm, weights=weights)
wls_results = wls_model.fit()

print("\n--- WLS Model Summary ---")
print(wls_results.summary())

wls_predictions = wls_results.predict(X_sm)

wls_r2 = r2_score(y, wls_predictions)
wls_mse = mean_squared_error(y, wls_predictions)

print(f"\nWLS R-squared: {wls_r2:.4f}")
print(f"WLS Mean Squared Error: {wls_mse:.4f}")

### Visualization of WLS Model Coefficients

This plot displays the magnitude and direction of the coefficients from our Weighted Least Squares (WLS) model. Features with larger absolute coefficient values have a stronger impact on the predicted Box-Cox transformed resale price, while the sign (positive or negative) indicates the direction of that impact.

In [ ]:
# Access the parameters table from the WLS results summary
wls_coefs = wls_results.summary2().tables[1]

# Re-extract non_zero_features (which are the selected features) based on the last ElasticNet run
# This line is copied from ZowSQy63YdpY to ensure `non_zero_features` is available
non_zero_features = X.columns[elastic.coef_ != 0]
selected_features = non_zero_features.tolist()

# Create feature names including the constant (intercept)
feature_names = ['const'] + selected_features

# Assign feature names to the index of the coefficients DataFrame
wls_coefs['Feature'] = feature_names
wls_coefs = wls_coefs.set_index('Feature')

# Filter out the 'const' (intercept) for visualization
coefs_to_plot = wls_coefs.drop('const')

# Sort coefficients by absolute value for better visualization
coefs_to_plot = coefs_to_plot.reindex(coefs_to_plot['Coef.'].abs().sort_values().index)

plt.figure(figsize=(12, 8))
sns.barplot(x='Coef.',
            y=coefs_to_plot.index,
            data=coefs_to_plot,
            palette='viridis')
plt.title('WLS Model Coefficients (Sorted by Magnitude)')
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

print("\n--- WLS Coefficients in detail ---")
print(coefs_to_plot[['Coef.', 'P>|t|']])

In [ ]:
feature_names = ['const'] + selected_features
wls_coefs = wls_results.summary2().tables[1]
wls_coefs['Feature'] = feature_names
wls_coefs = wls_coefs.set_index('Feature')

print("\n--- WLS Model Coefficients and P-values ---")
print(wls_coefs[['Coef.', 'P>|t|']])

### WLS Model: Residual Plot

In [ ]:
# Calculate residuals for the WLS model
wls_residuals = y.values.flatten() - wls_predictions.flatten()

plt.figure(figsize=(10, 8))
sns.scatterplot(x=wls_predictions, y=wls_residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Box-Cox Transformed Resale Price (WLS)')
plt.ylabel('Residuals (WLS)')
plt.title('Residual Plot (WLS Model)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### WLS Model: Scale-Location Plot (Homoscedasticity Check)

In [ ]:
sqrt_abs_wls_residuals = np.sqrt(np.abs(wls_residuals))

plt.figure(figsize=(10, 8))
sns.scatterplot(x=wls_predictions, y=sqrt_abs_wls_residuals, alpha=0.6)
plt.axhline(y=np.mean(sqrt_abs_wls_residuals), color='r', linestyle='--')
plt.xlabel('Predicted Box-Cox Transformed Resale Price (WLS)')
plt.ylabel('Square Root of Absolute Residuals (WLS)')
plt.title('Scale-Location Plot (WLS Homoscedasticity Check)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### WLS Model: Histogram and Q-Q Plot of Residuals

In [ ]:
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
sns.histplot(wls_residuals, kde=True)
plt.title('Histogram of Residuals (WLS)')
plt.xlabel('Residuals (WLS)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
stats.probplot(wls_residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals (WLS)')

plt.tight_layout()
plt.show()

## Linearity Assumption using Polynomial Features

### Add Polynomial Features to Top Predictors

In [ ]:
# Identify numerical columns for polynomial features
polynomial_features_cols = ['registered_year', 'max_power_clean_bhp', 'kms_driven', 'engine_capacity', 'mileage_number']

# Create a DataFrame for only these columns
X_poly_subset = Car_df[polynomial_features_cols]

# Initialize PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False) # include_bias=False to avoid duplicate constant term

# Fit and transform the subset
X_poly_transformed_subset = poly.fit_transform(X_poly_subset)

# Create a DataFrame from the transformed features
poly_feature_names = poly.get_feature_names_out(X_poly_subset.columns)
X_poly_df = pd.DataFrame(X_poly_transformed_subset, columns=poly_feature_names, index=Car_df.index)

# Drop original columns that were used to create polynomial features from Car_df (to avoid multicollinearity issues after adding squared terms of those columns)
Car_df_poly = Car_df.drop(columns=polynomial_features_cols)

# Concatenate the new polynomial features with the rest of the DataFrame
Car_df_poly = pd.concat([Car_df_poly, X_poly_df], axis=1)

display(Car_df_poly.head())

### Re-run Model Development with New Features

In [ ]:
# Redefine numerical_cols to include the new polynomial features and exclude the original ones
# Ensure we drop the target variable, the original resale_price, and registered_year_Imputation
numerical_cols_poly = Car_df_poly.select_dtypes(include=['float64','int64','int32']).drop(columns = ["boxcox_resale_price", "resale_price", "registered_year_Imputation", "price_per_km", "owner_rank"]).columns

scaler = StandardScaler()

X_poly = Car_df_poly[numerical_cols_poly]
X_poly_scaled = scaler.fit_transform(X_poly)

y = Car_df_poly[['boxcox_resale_price']]

# Lasso Feature Selection with Polynomial Features
lasso_poly = LassoCV(alphas=None, cv=5, random_state=42)
lasso_poly.fit(X_poly_scaled, y.values.ravel())

coef_df_poly = pd.Series(lasso_poly.coef_, index=numerical_cols_poly)
selected_features_poly = coef_df_poly[coef_df_poly != 0].index.tolist()
print("Selected features by Lasso (with Polynomial Features):", selected_features_poly)

# Linear Regression with Lasso-selected Polynomial Features
X_selected_poly = Car_df_poly[selected_features_poly]
X_selected_poly_scaled = scaler.fit_transform(X_selected_poly)

X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(X_selected_poly_scaled,
                                                                        y,
                                                                        test_size=0.3,
                                                                        random_state=42)
ml_poly = LinearRegression()
ml_poly.fit(X_train_poly, y_train_poly)
predictions_poly = ml_poly.predict(X_test_poly)

print("\nMean Squared Error of ML with Lasso (Polynomial Features): ", mean_squared_error(y_test_poly, predictions_poly))
print("R-Squared of ML with Lasso (Polynomial Features)", r2_score(y_test_poly, predictions_poly))

# WLS with Polynomial Features
X_sm_poly = sm.add_constant(X_selected_poly_scaled)

ols_model_poly = sm.OLS(y, X_sm_poly)
ols_results_poly = ols_model_poly.fit()

weights_poly = 1 / (np.abs(ols_results_poly.resid) + 1e-6)

wls_model_poly = sm.WLS(y, X_sm_poly, weights=weights_poly)
wls_results_poly = wls_model_poly.fit()

print("\n--- WLS Model Summary (Polynomial Features) ---")
print(wls_results_poly.summary())

wls_predictions_poly = wls_results_poly.predict(X_sm_poly)

wls_r2_poly = r2_score(y, wls_predictions_poly)
wls_mse_poly = mean_squared_error(y, wls_predictions_poly)

print(f"\nWLS R-squared (Polynomial Features): {wls_r2_poly:.4f}")
print(f"WLS Mean Squared Error (Polynomial Features): {wls_mse_poly:.4f}")

### Re-check Assumptions for WLS Model with Polynomial Features

In [ ]:
# Calculate residuals for the WLS model with polynomial features
wls_residuals_poly = y.values.flatten() - wls_predictions_poly.flatten()

plt.figure(figsize=(10, 8))
sns.scatterplot(x=wls_predictions_poly, y=wls_residuals_poly, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Box-Cox Transformed Resale Price (WLS, Poly)')
plt.ylabel('Residuals (WLS, Poly)')
plt.title('Residual Plot (WLS Model with Polynomial Features)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
sqrt_abs_wls_residuals_poly = np.sqrt(np.abs(wls_residuals_poly))

plt.figure(figsize=(10, 8))
sns.scatterplot(x=wls_predictions_poly, y=sqrt_abs_wls_residuals_poly, alpha=0.6)
plt.axhline(y=np.mean(sqrt_abs_wls_residuals_poly), color='r', linestyle='--')
plt.xlabel('Predicted Box-Cox Transformed Resale Price (WLS, Poly)')
plt.ylabel('Square Root of Absolute Residuals (WLS, Poly)')
plt.title('Scale-Location Plot (WLS Homoscedasticity Check with Polynomial Features)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
sns.histplot(wls_residuals_poly, kde=True)
plt.title('Histogram of Residuals (WLS, Poly)')
plt.xlabel('Residuals (WLS, Poly)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
stats.probplot(wls_residuals_poly, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals (WLS, Poly)')

plt.tight_layout()
plt.show()

Since weighted least squares still assumes a linear functional form and diagnostic tests indicated non-linearity, a Random Forest regression model was employed as a non-parametric alternative capable of capturing complex, non-linear relationships without restrictive assumptions.

## Model development (Random Forest Regression)

In [ ]:
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_selected_scaled,
                                                                y,
                                                                test_size=0.3,
                                                                random_state=42)
rf = RandomForestRegressor(
    n_estimators = 200,
    max_depth = None,
    min_samples_split = 2,
    min_samples_leaf = 1,
    max_features = 'sqrt',
    random_state = 42,
    n_jobs = -1
)

rf.fit(X_train_rf, y_train_rf)
predictions_rf = rf.predict(X_test_rf)

print("Mean Squared Error of ML with Random Forest: ", mean_squared_error(y_test_rf, predictions_rf))
print("R-Squared of ML with Random Forest", r2_score(y_test_rf, predictions_rf))

## Cross Validation Using K-Fold

In [ ]:
cv_r2_scores = cross_val_score(rf, X, y, cv=5, scoring='r2', n_jobs=-1)
print("CV R2 mean:", np.mean(cv_r2_scores))

cv_mse_scores = cross_val_score(rf, X, y, cv=5, scoring= 'neg_mean_squared_error', n_jobs=-1)
print("CV MSE:", np.mean(cv_mse_scores))

In [ ]:
plt.figure(figsize=(10, 7))
sns.scatterplot(x=y_test_rf.values.flatten(), y=predictions_rf, alpha=0.6)
plt.plot([min(y_test_rf.values.flatten()), max(y_test_rf.values.flatten())], [min(y_test_rf.values.flatten()), max(y_test_rf.values.flatten())], color='red', linestyle='--')
plt.title('Random Forest Regression: Actual vs. Predicted Box-Cox Resale Price')
plt.xlabel('Actual Box-Cox Transformed Resale Price')
plt.ylabel('Predicted Box-Cox Transformed Resale Price')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
rf_residuals = y_test_rf.values.flatten() - predictions_rf

plt.figure(figsize=(10, 7))
sns.scatterplot(x=predictions_rf, y=rf_residuals, alpha=0.6)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Random Forest Regression: Residual Plot')
plt.xlabel('Predicted Box-Cox Transformed Resale Price')
plt.ylabel('Residuals')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
sns.histplot(rf_residuals, kde=True)
plt.title('Histogram of Residuals (Random Forest)')
plt.xlabel('Residuals')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
stats.probplot(rf_residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals (Random Forest)')

plt.tight_layout()
plt.show()

## Multivariate Regression VS Random Forest Regression

In [ ]:
y_true_en = np.ravel(y_test_selected)
y_pred_en = np.ravel(ml.predict(X_test_selected))
res_en = y_true_en - y_pred_en

r2_en = r2_score(y_true_en, y_pred_en)
mse_en = mean_squared_error(y_true_en, y_pred_en)


y_true_rf = np.ravel(y_test_rf)
y_pred_rf = np.ravel(rf.predict(X_test_rf))
res_rf = y_true_rf - y_pred_rf

r2_rf = r2_score(y_true_rf, y_pred_rf)
mse_rf = mean_squared_error(y_true_rf, y_pred_rf)


metrics_df = pd.DataFrame(
    {
        "R2": [r2_en, r2_rf],
        "MSE": [mse_en, mse_rf],
    },
    index=["ElasticNet ML Linear", "Random Forest"],
)
display(metrics_df.style.format({"R2": "{:.4f}", "MSE": "{:.4f}"}))


### Residuals vs Predicted

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)
lim_res = max(np.max(np.abs(res_en)), np.max(np.abs(res_rf)))

sns.scatterplot(x=y_pred_en, y=res_en, alpha=0.6, ax=axes[0])
axes[0].axhline(0, color="red", linestyle="--", linewidth=1)
axes[0].set_title("ElasticNet ML Linear: Residuals vs Predicted")
axes[0].set_xlabel("Predicted (Box-Cox resale price)")
axes[0].set_ylabel("Residuals")
axes[0].set_ylim(-lim_res, lim_res)
axes[0].grid(True, linestyle="--", alpha=0.6)

sns.scatterplot(x=y_pred_rf, y=res_rf, alpha=0.6, ax=axes[1])
axes[1].axhline(0, color="red", linestyle="--", linewidth=1)
axes[1].set_title("Random Forest: Residuals vs Predicted")
axes[1].set_xlabel("Predicted (Box-Cox resale price)")
axes[1].set_ylim(-lim_res, lim_res)
axes[1].grid(True, linestyle="--", alpha=0.6)

plt.tight_layout()
plt.show()

### Residual distributions

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharex=True)
lim_hist = max(np.max(np.abs(res_en)), np.max(np.abs(res_rf)))

sns.histplot(res_en, bins=40, kde=True, ax=axes[0], color="#4c72b0")
axes[0].set_title("ElasticNet ML Linear: Residuals Distribution")
axes[0].set_xlabel("Residual")
axes[0].set_xlim(-lim_hist, lim_hist)
axes[0].grid(True, linestyle="--", alpha=0.6)

sns.histplot(res_rf, bins=40, kde=True, ax=axes[1], color="#55a868")
axes[1].set_title("Random Forest: Residuals Distribution")
axes[1].set_xlabel("Residual")
axes[1].set_xlim(-lim_hist, lim_hist)
axes[1].grid(True, linestyle="--", alpha=0.6)

plt.tight_layout()
plt.show()

## Statistical Summary: Linear Regression (ElasticNet) vs Random Forest

### Statistical Inference: ElasticNet Linear Regression Model

This section summarizes the key statistical findings for the ElasticNet Linear Regression model, providing insights into its overall fit, predictive performance, and adherence to assumptions.


In [ ]:
# ============================================================================
# STATISTICAL INFERENCE: ELASTICNET LINEAR REGRESSION MODEL
# ============================================================================

print("=" * 80)
print("STATISTICAL INFERENCE: ELASTICNET LINEAR REGRESSION MODEL")
print("=" * 80)

print("\n### 1. ANOVA Table")
print("**Interpretation**: The ANOVA table assesses the overall significance of the regression model.")
print("A statistically significant F-statistic (with a low p-value) indicates that the model, as a whole,")
print("explains a significant proportion of the variance in the dependent variable.\n")

# Calculate ANOVA components for Linear Regression
n = len(y_true_en)  # Number of observations
p = len(non_zero_features)  # Number of predictors

# Sum of Squares
y_mean = np.mean(y_true_en)
SS_total = np.sum((y_true_en - y_mean) ** 2)  # Total Sum of Squares
SS_regression = np.sum((y_pred_en - y_mean) ** 2)  # Regression Sum of Squares (Explained)
SS_residual = np.sum((y_true_en - y_pred_en) ** 2)  # Residual Sum of Squares (Unexplained)

# Degrees of Freedom
df_regression = p
df_residual = n - p - 1
df_total = n - 1

# Mean Squares
MS_regression = SS_regression / df_regression
MS_residual = SS_residual / df_residual

# F-statistic and p-value
F_statistic = MS_regression / MS_residual
p_value_F = 1 - stats.f.cdf(F_statistic, df_regression, df_residual)

# ANOVA Table
anova_lr = pd.DataFrame({
    'Source': ['Regression', 'Residual', 'Total'],
    'Sum of Squares': [SS_regression, SS_residual, SS_total],
    'df': [df_regression, df_residual, df_total],
    'Mean Square': [MS_regression, MS_residual, np.nan],
    'F-statistic': [F_statistic, np.nan, np.nan],
    'p-value': [p_value_F, np.nan, np.nan]
})

display(anova_lr.style.format({
    'Sum of Squares': '{:.4f}',
    'Mean Square': '{:.4f}',
    'F-statistic': '{:.4f}',
    'p-value': '{:.2e}'
}, na_rep='-').set_caption("ANOVA Table - Linear Regression Model"))

print("\n**Note**: This ANOVA table is based on the ElasticNet feature-selected linear regression model.")

# Model Significance
p_value_F_lr = p_value_F
model_significance_lr = ''
if p_value_F_lr < 0.001:
    model_significance_lr = '*** Highly Significant (p < 0.001)'
elif p_value_F_lr < 0.01:
    model_significance_lr = '** Significant (p < 0.01)'
elif p_value_F_lr < 0.05:
    model_significance_lr = '* Significant (p < 0.05)'
else:
    model_significance_lr = 'Not Significant'

# R-squared values
r_squared_train_lr = SS_regression / SS_total
adjusted_r_squared_train_lr = 1 - (SS_residual/df_residual) / (SS_total/df_total)

print(f"\n- **Model Significance**: {model_significance_lr}")
print(f"- **R-squared (Training)**: {r_squared_train_lr:.4f}")
print(f"- **Adjusted R-squared (Training)**: {adjusted_r_squared_train_lr:.4f}")

print(f"\n**Discussion**: The F-statistic of {F_statistic:.2f} with a p-value of {p_value_F:.2e}")
print(f"indicates that the model is {model_significance_lr.lower().replace('* ', '').replace(' (p < 0.001)', '')},")
print(f"meaning the predictors collectively have a significant linear relationship with the Box-Cox transformed")
print(f"resale price. The R-squared of {r_squared_train_lr:.4f} suggests that approximately {r_squared_train_lr*100:.2f}%")
print(f"of the variance in the transformed resale price can be explained by the model's independent variables.")

print("\n### 2. Performance Metrics (Test Set)")
print("**Interpretation**: These metrics evaluate the model's predictive accuracy on unseen data.\n")

# Calculating metrics
print(f"- R-squared (Test)**: {r2_en:.4f}")
print(f"- MSE (Test)**: {mse_en:.4f}")
print(f"- RMSE (Test)**: {np.sqrt(mse_en):.4f}")
print(f"- MAE (Test)**: {np.mean(np.abs(res_en)):.4f}")

print(f"\n**Discussion**: The R-squared value of {r2_en:.4f} on the test set indicates")
print(f"that the model generalizes reasonably well, explaining a substantial portion of the variance in the")
print(f"transformed resale price. The MSE and RMSE values suggest that the predictions are close to the actual values on average.")

print("\n### 3. Residual Statistics")
print("**Interpretation**: Analyzing residual statistics helps in evaluating the model's assumptions")
print("(e.g., normality of residuals, homoscedasticity).\n")

# Calculating residual statistics
print(f"- Mean Residual**: {np.mean(res_en):.4f}")
print(f"- Standard Deviation of Residuals**: {np.std(res_en):.4f}")
print(f"- Skewness of Residuals**: {stats.skew(res_en):.4f}")
print(f"- Kurtosis of Residuals**: {stats.kurtosis(res_en):.4f}")

print(f"\n**Discussion**: Ideally, the mean of the residuals should be close to zero, which is the case")
print(f"({np.mean(res_en):.4f}). However, the skewness of {stats.skew(res_en):.4f} and")
print(f"particularly high kurtosis of {stats.kurtosis(res_en):.4f} indicate that the residuals deviate")
print(f"significantly from a normal distribution, showing a heavy-tailed distribution with more outliers than")
print(f"a normal distribution. This suggests that while the model captures the central tendency well, there might")
print(f"be systematic errors or influential observations not fully accounted for, potentially violating the")
print(f"assumption of normally distributed errors. This non-normality was a driving factor for exploring")
print(f"non-linear models like Random Forest.")

print("\n" + "=" * 80)

### Statistical Inference: Random Forest Regression Model

This section summarizes the key statistical findings for the Random Forest Regression model, providing insights into its overall fit, predictive performance, and residuals.


In [ ]:
# ============================================================================
# STATISTICAL INFERENCE: RANDOM FOREST REGRESSION MODEL
# ============================================================================

print("=" * 80)
print("STATISTICAL INFERENCE: RANDOM FOREST REGRESSION MODEL")
print("=" * 80)

print("\n### 1. ANOVA Table")
print("**Interpretation**: The ANOVA table assesses the overall significance of the regression model.")
print("For Random Forest, this is an approximation as it is a non-parametric model. A statistically")
print("significant F-statistic (with a low p-value) suggests the model explains a significant proportion")
print("of the variance in the dependent variable.\n")

# Calculate ANOVA components for Random Forest
n_rf = len(y_true_rf)  # Number of observations
p_rf = len(non_zero_features)  # Number of features used

# Sum of Squares
y_mean_rf = np.mean(y_true_rf)
SS_total_rf = np.sum((y_true_rf - y_mean_rf) ** 2)  # Total Sum of Squares
SS_regression_rf = np.sum((y_pred_rf - y_mean_rf) ** 2)  # Regression Sum of Squares (Explained)
SS_residual_rf = np.sum((y_true_rf - y_pred_rf) ** 2)  # Residual Sum of Squares (Unexplained)

# Degrees of Freedom (for RF, effective df is approximate)
df_regression_rf = p_rf
df_residual_rf = n_rf - p_rf - 1
df_total_rf = n_rf - 1

# Mean Squares
MS_regression_rf = SS_regression_rf / df_regression_rf
MS_residual_rf = SS_residual_rf / df_residual_rf

# F-statistic and p-value
F_statistic_rf = MS_regression_rf / MS_residual_rf
p_value_F_rf = 1 - stats.f.cdf(F_statistic_rf, df_regression_rf, df_residual_rf)

# ANOVA Table
anova_rf = pd.DataFrame({
    'Source': ['Regression', 'Residual', 'Total'],
    'Sum of Squares': [SS_regression_rf, SS_residual_rf, SS_total_rf],
    'df': [df_regression_rf, df_residual_rf, df_total_rf],
    'Mean Square': [MS_regression_rf, MS_residual_rf, np.nan],
    'F-statistic': [F_statistic_rf, np.nan, np.nan],
    'p-value': [p_value_F_rf, np.nan, np.nan]
})

display(anova_rf.style.format({
    'Sum of Squares': '{:.4f}',
    'Mean Square': '{:.4f}',
    'F-statistic': '{:.4f}',
    'p-value': '{:.2e}'
}, na_rep='-').set_caption("ANOVA Table - Random Forest Model"))

print("\n**Note**: ANOVA for Random Forest is an approximation since RF is a non-parametric model.")

# Model Significance
model_significance_rf = ''
if p_value_F_rf < 0.001:
    model_significance_rf = '*** Highly Significant (p < 0.001)'
elif p_value_F_rf < 0.01:
    model_significance_rf = '** Significant (p < 0.01)'
elif p_value_F_rf < 0.05:
    model_significance_rf = '* Significant (p < 0.05)'
else:
    model_significance_rf = 'Not Significant'

# R-squared values
r_squared_train_rf = SS_regression_rf / SS_total_rf
adjusted_r_squared_train_rf = 1 - (SS_residual_rf/df_residual_rf) / (SS_total_rf/df_total_rf)

print(f"\n- **Model Significance**: {model_significance_rf}")
print(f"- **R-squared (Training)**: {r_squared_train_rf:.4f}")
print(f"- **Adjusted R-squared (Training)**: {adjusted_r_squared_train_rf:.4f}")

print(f"\n**Discussion**: The F-statistic of {F_statistic_rf:.2f} with a p-value of {p_value_F_rf:.2e}")
print(f"indicates that, as an approximation, the model is {model_significance_rf.lower().replace('* ', '').replace(' (p < 0.001)', '')},")
print(f"implying a significant relationship between predictors and the Box-Cox transformed resale price.")
print(f"The R-squared of {r_squared_train_rf:.4f} suggests that approximately {r_squared_train_rf*100:.2f}%")
print(f"of the variance in the transformed resale price is explained by the model's independent variables.")

print("\n### 2. Performance Metrics (Test Set)")
print("**Interpretation**: These metrics evaluate the model's predictive accuracy on unseen data.\n")

# Calculating metrics
print(f"- R-squared (Test)**: {r2_rf:.4f}")
print(f"- MSE (Test)**: {mse_rf:.4f}")
print(f"- RMSE (Test)**: {np.sqrt(mse_rf):.4f}")
print(f"- MAE (Test)**: {np.mean(np.abs(res_rf)):.4f}")

print(f"\n**Discussion**: The Random Forest model demonstrates strong predictive power with an R-squared of")
print(f"{r2_rf:.4f} on the test set, explaining a substantial proportion of the variance.")
print(f"The low MSE, RMSE, and MAE values suggest that the model's predictions are highly accurate and close to the actual values.")

print("\n### 3. Residual Statistics")
print("**Interpretation**: Analyzing residual statistics helps in evaluating the model's assumptions")
print("(e.g., normality of residuals, homoscedasticity).\n")

# Calculating residual statistics
print(f"- Mean Residual**: {np.mean(res_rf):.4f}")
print(f"- Standard Deviation of Residuals**: {np.std(res_rf):.4f}")
print(f"- Skewness of Residuals**: {stats.skew(res_rf):.4f}")
print(f"- Kurtosis of Residuals**: {stats.kurtosis(res_rf):.4f}")

print(f"\n**Discussion**: The mean residual of {np.mean(res_rf):.4f} is very close to zero, which is ideal.")
print(f"The skewness of {stats.skew(res_rf):.4f} and kurtosis of {stats.kurtosis(res_rf):.4f}")
print(f"indicate that the residuals are much closer to a normal distribution compared to the linear model,")
print(f"with less prominent tails and fewer outliers. This suggests better adherence to the assumption of")
print(f"normally distributed errors and a more robust fit.")

print("\n" + "=" * 80)

### ANOVA Comparison: Linear Regression vs Random Forest

In [ ]:
# ============================================================================
# SIDE-BY-SIDE ANOVA COMPARISON
# ============================================================================

print("=" * 80)
print("ANOVA COMPARISON: LINEAR REGRESSION vs RANDOM FOREST")
print("=" * 80)

anova_comparison = pd.DataFrame({
    'Metric': ['SS Regression (Explained)', 'SS Residual (Unexplained)', 'SS Total',
               'MS Regression', 'MS Residual', 'F-statistic', 'p-value',
               'R-squared', 'Adjusted R-squared', 'n (observations)', 'p (predictors)'],
    'Linear Regression': [
        SS_regression, SS_residual, SS_total,
        MS_regression, MS_residual, F_statistic, p_value_F,
        SS_regression/SS_total, 1 - (SS_residual/df_residual)/(SS_total/df_total),
        n, p
    ],
    'Random Forest': [
        SS_regression_rf, SS_residual_rf, SS_total_rf,
        MS_regression_rf, MS_residual_rf, F_statistic_rf, p_value_F_rf,
        SS_regression_rf/SS_total_rf, 1 - (SS_residual_rf/df_residual_rf)/(SS_total_rf/df_total_rf),
        n_rf, p_rf
    ]
})

anova_comparison['Difference'] = anova_comparison['Random Forest'] - anova_comparison['Linear Regression']
anova_comparison['Better Model'] = anova_comparison.apply(
    lambda row: 'Random Forest' if (
        (row['Metric'] in ['R-squared', 'Adjusted R-squared', 'F-statistic'] and row['Difference'] > 0) or
        (row['Metric'] in ['SS Residual (Unexplained)', 'MS Residual'] and row['Difference'] < 0)
    ) else 'Linear Regression' if (
        (row['Metric'] in ['R-squared', 'Adjusted R-squared', 'F-statistic'] and row['Difference'] < 0) or
        (row['Metric'] in ['SS Residual (Unexplained)', 'MS Residual'] and row['Difference'] > 0)
    ) else '-', axis=1
)

display(anova_comparison.style.format({
    'Linear Regression': lambda x: f'{x:.4f}' if isinstance(x, float) else str(x),
    'Random Forest': lambda x: f'{x:.4f}' if isinstance(x, float) else str(x),
    'Difference': lambda x: f'{x:+.4f}' if isinstance(x, float) else str(x)
}).set_caption("ANOVA Comparison Summary"))

# Variance Explained Comparison
print("\n--- Variance Explained ---")
print(f"Linear Regression explains {(SS_regression/SS_total)*100:.2f}% of the variance")
print(f"Random Forest explains {(SS_regression_rf/SS_total_rf)*100:.2f}% of the variance")
print(f"Difference: Random Forest explains {((SS_regression_rf/SS_total_rf) - (SS_regression/SS_total))*100:+.2f}% more variance")

In [ ]:
# ============================================================================
# STATISTICAL COMPARISON: Linear Regression (ElasticNet) vs Random Forest
# ============================================================================

print("=" * 80)
print("MODEL COMPARISON: LINEAR REGRESSION (ElasticNet) vs RANDOM FOREST")
print("=" * 80)

# --- 1. Performance Metrics Comparison ---
print("\n" + "=" * 80)
print("1. PERFORMANCE METRICS COMPARISON")
print("=" * 80)

comparison_df = pd.DataFrame({
    'Metric': ['R-squared (Test)', 'MSE (Test)', 'RMSE (Test)', 'MAE (Test)'],
    'Linear Regression (ElasticNet)': [
        r2_en,
        mse_en,
        np.sqrt(mse_en),
        np.mean(np.abs(res_en))
    ],
    'Random Forest': [
        r2_rf,
        mse_rf,
        np.sqrt(mse_rf),
        np.mean(np.abs(res_rf))
    ]
})
comparison_df['Difference (RF - LR)'] = comparison_df['Random Forest'] - comparison_df['Linear Regression (ElasticNet)']
comparison_df['Better Model'] = comparison_df.apply(
    lambda row: 'Random Forest' if (row['Metric'] == 'R-squared (Test)' and row['Difference (RF - LR)'] > 0) or
                                   (row['Metric'] != 'R-squared (Test)' and row['Difference (RF - LR)'] < 0)
                else 'Linear Regression', axis=1
)

display(comparison_df.style.format({
    'Linear Regression (ElasticNet)': '{:.6f}',
    'Random Forest': '{:.6f}',
    'Difference (RF - LR)': '{:+.6f}'
}).set_caption("Performance Metrics Comparison"))

# --- 2. Coefficients vs Feature Importances Comparison ---
print("\n" + "=" * 80)
print("2. COEFFICIENTS (LR) vs FEATURE IMPORTANCES (RF)")
print("=" * 80)

# Linear Regression Coefficients (standardized)
lr_coefs = pd.DataFrame({
    'Feature': non_zero_features.tolist(),
    'LR_Coefficient': ml.coef_.flatten()
})
lr_coefs['LR_Abs_Coef'] = lr_coefs['LR_Coefficient'].abs()
lr_coefs['LR_Normalized'] = lr_coefs['LR_Abs_Coef'] / lr_coefs['LR_Abs_Coef'].sum()

# Random Forest Feature Importances
rf_importance = pd.DataFrame({
    'Feature': non_zero_features.tolist(),
    'RF_Importance': rf.feature_importances_
})
rf_importance['RF_Normalized'] = rf_importance['RF_Importance'] / rf_importance['RF_Importance'].sum()

# Merge for comparison
feature_comparison = lr_coefs.merge(rf_importance, on='Feature')
feature_comparison = feature_comparison.sort_values('RF_Importance', ascending=False)

print(f"\n{'Feature':<35} {'LR Coef':>12} {'LR Norm%':>10} {'RF Imp':>12} {'RF Norm%':>10}")
print("-" * 79)
for _, row in feature_comparison.iterrows():
    print(f"{row['Feature']:<35} {row['LR_Coefficient']:>12.4f} {row['LR_Normalized']*100:>9.2f}% {row['RF_Importance']:>12.4f} {row['RF_Normalized']*100:>9.2f}%")

# --- 3. Residual Statistics Comparison ---
print("\n" + "=" * 80)
print("3. RESIDUAL STATISTICS COMPARISON")
print("=" * 80)

residual_stats = pd.DataFrame({
    'Statistic': ['Mean', 'Std Dev', 'Min', '25%', 'Median', '75%', 'Max', 'Skewness', 'Kurtosis'],
    'Linear Regression': [
        np.mean(res_en), np.std(res_en), np.min(res_en),
        np.percentile(res_en, 25), np.median(res_en), np.percentile(res_en, 75),
        np.max(res_en), stats.skew(res_en), stats.kurtosis(res_en)
    ],
    'Random Forest': [
        np.mean(res_rf), np.std(res_rf), np.min(res_rf),
        np.percentile(res_rf, 25), np.median(res_rf), np.percentile(res_rf, 75),
        np.max(res_rf), stats.skew(res_rf), stats.kurtosis(res_rf)
    ]
})

display(residual_stats.style.format({
    'Linear Regression': '{:.6f}',
    'Random Forest': '{:.6f}'
}).set_caption("Residual Statistics Comparison"))

print("\n" + "=" * 80)

### Coefficient (LR) vs Feature Importance (RF) Visualization

In [ ]:
# ============================================================================
# VISUALIZATION: COEFFICIENT vs FEATURE IMPORTANCE COMPARISON
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 10))

# Sort by RF importance for consistent ordering
feature_comparison_sorted = feature_comparison.sort_values('RF_Importance', ascending=True)

# Plot 1: Linear Regression Coefficients (Normalized Absolute Values)
ax1 = axes[0]
colors_lr = ['#e74c3c' if c < 0 else '#3498db' for c in feature_comparison_sorted['LR_Coefficient']]
ax1.barh(feature_comparison_sorted['Feature'], feature_comparison_sorted['LR_Normalized'] * 100, color=colors_lr)
ax1.set_xlabel('Normalized Importance (%)')
ax1.set_title('Linear Regression\n(Normalized |Coefficients|)', fontsize=12, fontweight='bold')
ax1.grid(axis='x', linestyle='--', alpha=0.7)

# Plot 2: Random Forest Feature Importances
ax2 = axes[1]
ax2.barh(feature_comparison_sorted['Feature'], feature_comparison_sorted['RF_Normalized'] * 100, color='#27ae60')
ax2.set_xlabel('Normalized Importance (%)')
ax2.set_title('Random Forest\n(Feature Importances)', fontsize=12, fontweight='bold')
ax2.grid(axis='x', linestyle='--', alpha=0.7)

plt.suptitle('Feature Importance Comparison: Linear Regression vs Random Forest', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Correlation between LR coefficients and RF importances
correlation = feature_comparison['LR_Abs_Coef'].corr(feature_comparison['RF_Importance'])
print(f"\nCorrelation between |LR Coefficients| and RF Feature Importances: {correlation:.4f}")